In [ ]:
import unittest
import torch
import math
import os
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.tensorboard import SummaryWriter
from platform import python_version
import torchvision.models as models

class TestPytorchNotebook(unittest.TestCase):
    
    def test_python_version(self):
        expected_major_minor = '3.11'  # Set the expected version (x.y)
        actual_major_minor = '.'.join(python_version().split('.')[:2]) 
        self.assertEqual(actual_major_minor, expected_major_minor, "incorrect version")
    
    def test_torch_version(self):
        expected_major_minor = '2.4'  # Set the expected version (x.y)
        actual_major_minor = '.'.join(torch.__version__.split('.')[:2]) 
        self.assertEqual(actual_major_minor, expected_major_minor, "incorrect version")
        
    def test_tensor_creation(self):
        # Test tensor creation
        tensor = torch.tensor([1, 2, 3])
        self.assertIsInstance(tensor, torch.Tensor)
        
    def test_tensor_operations(self):
        # Test tensor operations
        a = torch.tensor([1, 2, 3])
        b = torch.tensor([4, 5, 6])
        c = a + b
        self.assertTrue(torch.all(torch.eq(c, torch.tensor([5, 7, 9]))))
        
    def test_tensor_shape(self):
        # Test tensor shape
        tensor = torch.tensor([[1, 2, 3], [4, 5, 6]])
        self.assertEqual(tensor.shape, (2, 3))
        
    def test_polynomial_fitting(self):
        dtype = torch.float
        device = torch.device("cpu")
        # device = torch.device("cuda:0") # Uncomment this to run on GPU

        # Create random input and output data
        x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
        y = torch.sin(x)

        # Randomly initialize weights
        a = torch.randn((), device=device, dtype=dtype)
        b = torch.randn((), device=device, dtype=dtype)
        c = torch.randn((), device=device, dtype=dtype)
        d = torch.randn((), device=device, dtype=dtype)

        learning_rate = 1e-6
        for t in range(2000):
            # Forward pass: compute predicted y
            y_pred = a + b * x + c * x ** 2 + d * x ** 3

            # Compute and print loss
            loss = (y_pred - y).pow(2).sum().item()
            if t % 100 == 99:
                print(t, loss)

            # Backprop to compute gradients of a, b, c, d with respect to loss
            grad_y_pred = 2.0 * (y_pred - y)
            grad_a = grad_y_pred.sum()
            grad_b = (grad_y_pred * x).sum()
            grad_c = (grad_y_pred * x ** 2).sum()
            grad_d = (grad_y_pred * x ** 3).sum()

            # Update weights using gradient descent
            a -= learning_rate * grad_a
            b -= learning_rate * grad_b
            c -= learning_rate * grad_c
            d -= learning_rate * grad_d


        print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')
        
    def test_image_transform(self):
            # Define a sample image
            sample_image = Image.new('RGB', (100, 100), color='red')

            # Define a transformation pipeline
            transform = transforms.Compose([
                transforms.Resize((64, 64)),
                transforms.ToTensor()
            ])

            # Apply the transformation
            transformed_image = transform(sample_image)

            # Check if the transformed image is a PyTorch tensor
            self.assertTrue(isinstance(transformed_image, torch.Tensor))

            # Check if the transformed image has the correct shape
            self.assertEqual(transformed_image.shape, (3, 64, 64))  # Assuming RGB image

    def test_tensorboard_integration(self):
            # Define a sample image
            sample_image = Image.new('RGB', (100, 100), color='red')

            # Define a transformation pipeline
            transform = transforms.Compose([
                transforms.Resize((64, 64)),
                transforms.ToTensor()
            ])

            # Apply the transformation
            transformed_image = transform(sample_image)

            # Initialize TensorBoard writer
            writer = SummaryWriter(log_dir="./logs")

            # Add the transformed image to TensorBoard
            writer.add_image("Transformed Image", transformed_image, global_step=0)

            # Close the writer
            writer.close()

            # You can manually check the generated TensorBoard logs to verify the image

            # Assert that the writer has been successfully created and used
            self.assertTrue(isinstance(writer, SummaryWriter))
            
    def test_convert_to_onnx(self):
        # Load a pre-trained PyTorch model (for example, ResNet-18)
        model = models.resnet18(pretrained=True)
        model.eval()

        # Create a sample input tensor with the appropriate shape
        input_tensor = torch.randn(1, 3, 224, 224)  # (batch_size, channels, height, width)

        # Export the model to ONNX format
        onnx_path = "resnet18.onnx"
        torch.onnx.export(model, input_tensor, onnx_path, verbose=True)

        # Check if the ONNX file exists
        self.assertTrue(os.path.exists(onnx_path), f"ONNX file {onnx_path} not found")
        
suite = unittest.TestLoader().loadTestsFromTestCase(TestPytorchNotebook)
unittest.TextTestRunner().run(suite)